In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

#class knot

def random_knot(grid_size = 10, distribution='uniform'):
    #row_list = list(range(grid_size))
    #x_row_choice = []
    #column_list = list(range(1,grid_size))
    #o_column_choice = []
    #while len(column_list) != 0:
        #new_x_row_choice = random.choice(row_list)
        #row_list.remove(new_x_row_choice)
        #x_row_choice.append(new_x_row_choice)
        #new_o_column_choice = random.choice(column_list)
        #column_list.remove(new_o_column_choice)
        #o_column_choice.append(new_o_column_choice)
    #x_row_choice.append(row_list[0])
    #o_column_choice.append(0)
    x_rows = np.random.permutation(grid_size)
    o_columns = np.random.permutation(np.arange(1,grid_size))
    o_columns = np.append(o_columns,0)
    return (x_rows, o_columns)

def crossing_number(knot):
    x_rows = knot[0]
    o_columns = knot[1]
    grid_size = len(x_rows)
    passing_matrix = np.zeros((grid_size,grid_size))
    crossing_counter = 0
    for i in range(grid_size):
        #x_coordinates = (x_rows[i], o_columns[i-1])
        #o_coordinates_vert = (x_rows[i-1], o_columns[i-1])
        #o_coordinates_horiz = (x_rows[i], o_columns[i])
        min_vert = min(x_rows[i]+1,x_rows[i-1])
        max_vert = max(x_rows[i],x_rows[i-1]+1)
        min_horiz = min(o_columns[i-1],o_columns[i]+1)
        max_horiz = max(o_columns[i-1]+1,o_columns[i])
        for j in range(min_vert,max_vert):
            if passing_matrix[j,o_columns[i-1]] != 0:
                crossing_counter += 1
            passing_matrix[j,o_columns[i-1]] += 1
        for j in range(min_horiz,max_horiz):
            if passing_matrix[x_rows[i],j] != 0:
                crossing_counter += 1
            passing_matrix[x_rows[i],j] += 1
        #print(passing_matrix)
    return crossing_counter

def knot_matrix(knot):
    x_rows = knot[0]
    o_columns = knot[1]
    grid_size = len(x_rows)
    matrix = np.zeros((grid_size,grid_size))
    for i in range(grid_size):
        x_cell = (x_rows[i], o_columns[i-1])
        o_cell = (x_rows[i-1], o_columns[i-1])
        matrix[x_cell] = 1
        matrix[o_cell] = -1
    return matrix


In [2]:
import tensorflow as tf

def init_weights(shape, init_method='xavier', xavier_params = (None, None)):
    if init_method == 'zeros':
        return tf.Variable(tf.zeros(shape, dtype=tf.float32))
    elif init_method == 'uniform':
        return tf.Variable(tf.random_normal(shape, stddev=0.01, dtype=tf.float32))
    else: #xavier
        (fan_in, fan_out) = xavier_params
        low = -4*np.sqrt(6.0/(fan_in + fan_out)) # {sigmoid:4, tanh:1} 
        high = 4*np.sqrt(6.0/(fan_in + fan_out))
        return tf.Variable(tf.random_uniform(shape, minval=low, maxval=high, dtype=tf.float32))


In [3]:
def generate_batch(grid_size=10, batch_size=100, representation = 'matrix'):
    knot = random_knot(grid_size)
    crossings = crossing_number(knot)
    if representation == 'matrix':
        knot = knot_matrix(knot).flatten()
    if representation == 'path_choice':
        knot = np.append(knot[0],knot[1])
    batch_knots = np.array([knot])
    batch_crossings = np.array([[crossings]])

    for _ in range(batch_size-1):
        knot = random_knot(grid_size)
        crossings = crossing_number(knot)
        if representation == 'matrix':
            knot = knot_matrix(knot).flatten()
        if representation == 'path_choice':
            knot = np.append(knot[0],knot[1])
        batch_knots = np.append(batch_knots,[knot],0)
        batch_crossings = np.append(batch_crossings, [[crossings]], 0)
    
    return {'knot':batch_knots, 'crossings':batch_crossings}

generate_batch(grid_size=3, batch_size = 5)

{'crossings': array([[0],
        [0],
        [1],
        [1],
        [1]]), 'knot': array([[-1.,  0.,  1.,  0.,  1., -1.,  1., -1.,  0.],
        [ 1.,  0., -1.,  0., -1.,  1., -1.,  1.,  0.],
        [-1.,  1.,  0.,  1.,  0., -1.,  0., -1.,  1.],
        [ 0., -1.,  1.,  1.,  0., -1., -1.,  1.,  0.],
        [ 0., -1.,  1.,  1.,  0., -1., -1.,  1.,  0.]])}

In [ ]:
x = tf.Constant()

In [5]:
import tensorflow as tf

import time


grid_size = 15
hidden_layer_mult = 7
layer_num = 2
num_hidden = grid_size*hidden_layer_mult

#optimizer_coefficient = 1e-1

train_batch_size = 3

valid_size = 200

global_epochs = 1
exec_time = 1

errors = np.zeros(exec_time*6 + 1)

for j in range(global_epochs):
    
    print("Starting global epoch %d" %j)
    
    sess = tf.InteractiveSession()
    knot = tf.placeholder(tf.float32, shape=[None, grid_size**2], name='knot')
    crossings = tf.placeholder(tf.float32, shape=[None,1], name='crossings')    

    if layer_num > 0:
        w_h_1 = init_weights([grid_size**2, num_hidden], 'xavier', xavier_params=(1, num_hidden))
        b_h_1 = init_weights([num_hidden], 'zeros')
        h_1 = tf.nn.relu(tf.matmul(knot, w_h_1) + b_h_1)
    
    if layer_num > 1:
        w_h_2 = init_weights([num_hidden, num_hidden], 'xavier', xavier_params=(1, num_hidden))
        b_h_2 = init_weights([num_hidden], 'zeros')
        h_2 = tf.nn.relu(tf.matmul(h_1, w_h_2) + b_h_2)
        
    if layer_num > 2:
        w_h_3 = init_weights([num_hidden, num_hidden], 'xavier', xavier_params=(1, num_hidden))
        b_h_3 = init_weights([num_hidden], 'zeros')
        h_3 = tf.nn.relu(tf.matmul(h_2, w_h_3) + b_h_3)
        

    w_o = init_weights([num_hidden, 1], 'xavier', xavier_params=(num_hidden, 1))
    b_o = init_weights([1], 'zeros')
    
    if layer_num == 1:
        crossings_ = tf.matmul(h_1, w_o) + b_o
    if layer_num == 2:
        crossings_ = tf.matmul(h_2, w_o) + b_o
    if layer_num == 3:
        crossings_ = tf.matmul(h_3, w_o) + b_o    

    #train_step = tf.train.GradientDescentOptimizer(optimizer_coefficient).minimize(loss)
    train_step = tf.train.AdamOptimizer().minimize(tf.nn.l2_loss(crossings_ - crossings))

    sess.run(tf.global_variables_initializer())

    i=0
    num_cycle=0
    t_end = time.time() + 60 * exec_time
    t_start = time.time()

    while time.time() < t_end:

        batch = generate_batch(grid_size=grid_size, batch_size = train_batch_size, representation = 'matrix')
        sess.run(train_step, feed_dict={knot: batch['knot'], crossings: batch['crossings']})
        num_cycle += 1
        
        if time.time() - t_start > 10*i: 
            
            valid_batch = generate_batch(grid_size=grid_size, batch_size = valid_size, representation = 'matrix')
            mse = sess.run(tf.nn.l2_loss(crossings_ - valid_batch['crossings']),  feed_dict={knot:valid_batch['knot']})
            
            print ("after %d seconds, knots tested %d, MSE %d " %(int(time.time() - t_start), 
                                                            num_cycle*train_batch_size, int(mse)))
            if i>2:
                errors[i] = errors[i]*(j/(j+1)) + mse/(j+1)
            i+=1
    print(" ")

plt.plot(errors)
plt.xlabel('seconds/10')
plt.ylabel('MSE')

Starting global epoch 0
after 0 seconds, knots tested 3, MSE 109547 
after 10 seconds, knots tested 14007, MSE 8263 
after 20 seconds, knots tested 28656, MSE 5030 
after 30 seconds, knots tested 43563, MSE 5169 
after 40 seconds, knots tested 58257, MSE 5869 


KeyboardInterrupt: 

In [ ]:
import tensorflow as tf

import time


grid_size = 10
hidden_layer_mult = 4
num_hidden = grid_size*hidden_layer_mult

optimizer_coefficient = 1e-1

train_batch_size = 25
train_steps = 200
valid_size = 200

global_epochs = 1
exec_time = 3

errors = np.zeros(exec_time*6 + 1)

for j in range(global_epochs):
    
    print("Starting global epoch %d" %j)
    
    sess = tf.InteractiveSession()
    knot = tf.placeholder(tf.float32, shape=[None, 2*grid_size], name='knot')
    crossings = tf.placeholder(tf.float32, shape=[None,1], name='crossings')    

    w_h = init_weights([2*grid_size, num_hidden], 'xavier', xavier_params=(1, num_hidden))
    b_h = init_weights([num_hidden], 'zeros')
    h = tf.nn.relu(tf.matmul(knot, w_h) + b_h)

    w_o = init_weights([num_hidden, 1], 'xavier', xavier_params=(num_hidden, 1))
    b_o = init_weights([1], 'zeros')

    crossings_ = tf.matmul(h, w_o) + b_o

    #train_step = tf.train.GradientDescentOptimizer(optimizer_coefficient).minimize(loss)
    train_step = tf.train.AdamOptimizer().minimize(tf.nn.l2_loss(crossings_ - crossings))

    sess.run(tf.global_variables_initializer())

    i=0
    t_end = time.time() + 60 * exec_time
    t_start = time.time()

    while time.time() < t_end:
        for _ in range(train_steps):
            batch = generate_batch(grid_size=grid_size, batch_size = train_batch_size)
            sess.run(train_step, feed_dict={knot: batch['knot'], crossings: batch['crossings']})
        valid_batch = generate_batch(grid_size=grid_size, batch_size = valid_size)
        mse = sess.run(tf.nn.l2_loss(crossings_ - valid_batch['crossings']),  feed_dict={knot:valid_batch['knot']})

        if time.time() - t_start > 10*i: 
            print ("after %d seconds, validation MSE %d" %(int(time.time() - t_start), int(mse)))
            if i>5:
                errors[i] = errors[i]*(j/(j+1)) + mse/(j+1)
            i+=1
    print(" ")

plt.plot(errors)
plt.xlabel('seconds/10')
plt.ylabel('MSE')